In [10]:
!pip install tensorflow_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
import os
import pathlib
import joblib

import numpy as np
import matplotlib.pyplot as plt

import cv2
import PIL

import tensorflow as tf
import tensorflow_hub as hub

In [12]:
data_dir = pathlib.Path("/content/drive/MyDrive/Data sets/Plant_leave_disease_dataset_with_augmentation/Apple")

In [13]:
class_images = {
    "scab": list(data_dir.glob("Apple___Apple_scab/*")), 
    "black_rot": list(data_dir.glob("Apple___Black_rot/*")),
    "rust": list(data_dir.glob("Apple___Cedar_apple_rust/*")),
    "healthy": list(data_dir.glob("Apple___healthy/*"))
}

class_labels = {
    "scab": 0,
    "black_rot": 1,
    "rust": 2,
    "healthy": 3
}

In [14]:
X, y = [], []

for image_name, images in class_images.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img, (224, 224))
        X.append(resized_img)
        y.append(class_labels[image_name])
        
X = np.array(X)
y = np.array(y)

Load the mobilenetv2 model, + (3, ) adds a third dimension.

In [15]:
feature_extractor_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

pretrained_model_without_top_layer = hub.KerasLayer(
    feature_extractor_model, input_shape=(224, 224, 3), trainable=False)

In [16]:
model = tf.keras.Sequential([
  pretrained_model_without_top_layer,
  tf.keras.layers.Dense(4)
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 1280)              2257984   
                                                                 
 dense_1 (Dense)             (None, 4)                 5124      
                                                                 
Total params: 2,263,108
Trainable params: 5,124
Non-trainable params: 2,257,984
_________________________________________________________________


In [17]:
model.compile(
  optimizer="adam",
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['acc'])

model.fit(X, y, epochs=10)

Epoch 1/10
146/146 [==============================] - 9s 40ms/step - loss: 0.7180 - acc: 0.7229
Epoch 2/10
146/146 [==============================] - 5s 36ms/step - loss: 0.4113 - acc: 0.8596
Epoch 3/10
146/146 [==============================] - 5s 36ms/step - loss: 0.3372 - acc: 0.8844
Epoch 4/10
146/146 [==============================] - 5s 36ms/step - loss: 0.2992 - acc: 0.8984
Epoch 5/10
146/146 [==============================] - 5s 36ms/step - loss: 0.2738 - acc: 0.9059
Epoch 6/10
146/146 [==============================] - 5s 36ms/step - loss: 0.2486 - acc: 0.9173
Epoch 7/10
146/146 [==============================] - 5s 36ms/step - loss: 0.2306 - acc: 0.9210
Epoch 8/10
146/146 [==============================] - 5s 36ms/step - loss: 0.2183 - acc: 0.9294
Epoch 9/10
146/146 [==============================] - 5s 36ms/step - loss: 0.2055 - acc: 0.9311
Epoch 10/10
146/146 [==============================] - 5s 36ms/step - loss: 0.1943 - acc: 0.9363


In [18]:
predicted = model.predict(np.array([X[200]]))
predicted = np.argmax(predicted, axis=1)

1/1 [==============================] - 1s 698ms/step


In [20]:
predicted

array([3])

In [23]:
joblib.dump(model, "/content/drive/MyDrive/Colab Notebooks/Plant disease detection/apple_model.joblib")

['/content/drive/MyDrive/Colab Notebooks/Plant disease detection/apple_model.joblib']